# `learning-docker` using Docker Desktop in Windows

## Check if `docker` is installed properly

In [ ]:
!docker info

In [ ]:
!docker run hello-world

## The Docker flow

In [ ]:
!docker images

A docker image has just enough of the operating system to do what is needed for code to be ran. `docker images` shows all of the images that have been created. `docker rmi <IMAGE ID>` will remove the installed image.

The `docker run` command takes an image and turns in into a container with a running process in it.

In [ ]:
!docker run -ti ubuntu:latest bash 

`-ti` is terminal interactive and is used to have a full terminal within the container so that you can run the shell and get things like tab completion and formatting to work correctly.

In [ ]:
!docker ps -l

Run `docker ps` to get a list of running docker images and run `docker ps -l` to get the latest image that was ran.

Containers that are running are unique and independent of other containers that are running based on the same image. If one creates a container and adds a file to it, another operating container would not be able to see the new image. 

For a web-based application, you can have one container that holds the MongoDB database, another container that holds the React front-end, and the hosting server in the final container. Volumes contain the data for the containers. Container networking allows them to communicate with each other.

To see all containers (including stopped containers), run `docker ps -a` and `docker ps -l` shows the last container to exit (mentioned above).

The `docker commit` command takes containers and makes images out of them. `docker run` and `docker commit` are complementary to each other.

The `latest` tag is optional in docker. `docker run -ti ubuntu bash` is the same to `docker run -ti ubuntu:latest bash`.

In [ ]:
!docker ps -l

In [ ]:
!docker commit 11d30f7d8158

We now have a new image. The original is unchanged. We now need to tag the image.

In [ ]:
!docker tag dfca65088a629cb623abb71717cee366a52083eab2e0a03a75d4cce09e425b59 my-image

In [ ]:
!docker images

We can skip `docker tag` by doing `docker commit stupefied_poitras my-image-2`.

In [ ]:
!docker commit stupefied_poitras my-image-2

In [ ]:
!docker images

## Running processes in containers

`docker run --rm -d`
- Containers have a main process
- The container stops when the process stops
- Containers have names
- `--rm` if you want to run something in a container but don't want to keep it after. Same as `docker rm <container name>`.
- `-d` for detached. It starts the container and leaves it running in the background. Run `docker attach <container name>` to connect to the running container. `CTRL+P` + `CTRL-Q` to detach in a running container, but leaves it running.

If you've started a container and it already has something running, `docker exec` adds another process to a running container. It's great for debugging and DB administration, but you can't add ports, volumes, etc.

`docker exec -ti <already running container name> <process to add>`

## Managing containers

`docker logs` (Docker keeps the logs of the container around as long as you keep the container around)
- view the output of containers
- `docker logs <container name>`

In [ ]:
!docker run --name example -d ubuntu bash -c "lose /etc/password"

In [ ]:
!docker logs example

Killing and removing containers
- `docker kill <container name>` (makes it stop)
- `docker rm <container name>` (makes it be gone)

Resource Contraints
- Memory limits: `docker run --memory <max. allowed memory> <container name> <process>`
- CPU limits: `docker run --cpu-shares <relative to other containers>`, `docker run --cpu-quota <general limitations>` 

Lessons learned
- Don't let containers fetch their dependencies when they start
- Don't leave important things in unnamed stopped containers

## Managing images

`docker images` lists downloaded images. It doesn't list images that are able to get be downloaded. Images that share underlying data don't repeat the data itself; docker is space efficient.

In [ ]:
!docker ps -l 

In [ ]:
!docker commit f233f2ac34d6 my-image-14:1.0.0

In [ ]:
!docker images

A common naming convention of tagging images: `registry.example.com:port/organization/image-name:version-tag`

Images are grabbed by using `docker pull`, but this is done automatically by `docker run`. The opposite of `docker pull` is `docker push`. `docker rmi image:tag` removes an image from the system.

## Volumes

Volumes are virtual "discs" to store and share data from container-container (`volumes-from`) and container-host (`-v`). Volumes are not part of images.
Two main varieties:
- Presistent (container-container)
- Ephemeral (container-host)

## Registries
- Registries manage and distribute images
- Docker (the company) offers these for free
- One can run their own, as well

In [ ]:
!docker search ubuntu

`docker hub` for containers is extremely similar to `pip` and `git` for software.

## Building Docker images

A `Dockerfile` is a small program to create an image. It is ran with `docker build -t <name of Dockerfile> .`. Each line in a Dockerfile is its own call to `docker run`. Put the most volatile steps at the end of the file since each step is cached to avoid running steps too often. 

In [ ]:
%%file Dockerfile

FROM busybox
RUN echo "building simple docker image."
CMD echo "hello, world."

In [ ]:
!docker build -t hello .

In [ ]:
!docker run --rm hello

*.dockerignore* files can help similarly to *.gitignore*.*

An example *.dockerignore* file
```
node_modules
npm-debug.log
```

An additional *Dockerfile* example
```
FROM node

WORKDIR /usr/src/app

COPY package*.json ./

RUN npm install

COPY . .

EXPOSE 4000
CMD [ "npm", "start"]
```

In [ ]:
!docker run -p 4000:4000 <IMAGE NAME> # Runs the node image example above.

In [ ]:
!docker stop <CONTAINER ID> # Stops the running container.

In [ ]:
!docker start <CONTAINER ID> # Re-starts the stopped container.

`docker-compose` allows one to manage multiple containers with a single file. It lets docker know which services we want to compose. It's essentially a way to replace the `docker run` commands with a single file.

In [ ]:
%%file docker-compose.yml

app:
  container_name: app
  restart: always
  build: .
  ports:
    - "4000:4000"
  links:
    - mongo
mongo:
  container_name: mongo
  image: mongo
  expose:
    - "27017"
  volumes:
    - ./data:/data/db
  ports:
    - "27017:27017"

Run `docker-compose build` for the file above. Then run `docker-compose up -d mongo` since we want the mongo container to run first. To make sure that it's running, run `docker logs <CONTAINER ID>`. Then run `docker-compose up -d app` to run the main app.

Docker can be integrated with a continuous integration framework. One can use something like Travis CI using the following file.

```yaml
sudo: required
services:
  - docker

script:
  - docker build -t <dockeruser/nameofproject> .
  - docker images <dockeruser/nameofproject>

before_deploy:
  - docker login -u <username>  -p <userpassword>

deploy:
  provider: script
  script: docker push <dockeruser/nameofproject>
  on:
    branch: master
```

## Kubernetes

- Containers run programs
- Pods group containers together
- Services make pods available to others
- Labels are used for very advanced service discovery
- `kubectl` makes scripting large operations possible (ex. `kubectl get services -o wide`)
- Very flexible overlay networking
- Runs equally well on your hardware or a cloud provider
- Built-in service discovery
- `EC2 Container Service (ECS)` is another possible orchestration system